# Столбцовая перестановка

In [1]:
import math
import numpy as np
import string

# русский алфавит
abc  = [chr(code) for code in range(ord('а'), ord('я') + 1)] 

# словарь вида {буква : порядковый номер}
letter2number = {abc[i] : i for i in range(len(abc))} 

In [2]:
def columnar_cipher(message, key):
    """
    Шифрует сообщение message методом столбцовой перестановки с ключом key
    """
    n = len(key) # длина ключа aka число столбцов

    # приводим сообщение к нижнему регистру, избавляемся от пробелов и знаков препинания
    mes = message.lower().replace(" ", "")
    mes = mes.translate(str.maketrans('', '', string.punctuation))

    m = math.ceil(len(mes) / n) # число строк

    # таблица для записи сообщения; лишние ячейки заполняются мусором - буквами "а"
    table = np.full((m, n), 'а') 

    for i in range(m): # проходим по всем строкам..
        for j in range(n): # и столбцам:
            if i * n + j < len(mes): # если в сообщении ещё есть незаписанные символы
                table[i][j] = mes[i * n + j] # записываем очередную букву в таблицу
            else: # иначе
                break # выходим из цикла
    
    # порядковые номера букв в ключевом слове, отстортированные по возрастанию
    nums = sorted([letter2number[letter] for letter in key])

    # меняем номера обратно на буквы..
    route_order = [abc[number] for number in nums]
    # и получаем порядок, в котором нужно брать столбцы
    route_order = [key.index(letter) for letter in route_order]

    # криптограмма
    message_encrypted = ""

    for j in route_order: # проходим по столбцам в заданном порядке
        for i in range(m): # проходим по всем строкам
            message_encrypted += table[i][j] # добавляем очередную букву в криптограмму

    return message_encrypted

In [3]:
print(columnar_cipher("Нельзя недооценивать противника", "пароль"))
print(columnar_cipher("Стремясь к лучшему, мы часто портим хорошее", "корольлир"))

еенпнзоатаьовокннеьвлдирияцтиа
ьмреслчимеормеортуамтуамрчсхрчсхямпо


# Таблица Виженера

In [4]:
# составим таблицу Виженера для русского алфавита
vigenere_table = np.array(abc) # первая строчка - сам алфавит

for i in range(1, len(abc)):
    # получаем очередную строку, 
    # сдвигая алфавит на i позиций
    row = np.roll(abc, -i) 
    # добавляем строку к таблице
    vigenere_table = np.vstack((vigenere_table, row))

vigenere_table

array([['а', 'б', 'в', ..., 'э', 'ю', 'я'],
       ['б', 'в', 'г', ..., 'ю', 'я', 'а'],
       ['в', 'г', 'д', ..., 'я', 'а', 'б'],
       ...,
       ['э', 'ю', 'я', ..., 'ъ', 'ы', 'ь'],
       ['ю', 'я', 'а', ..., 'ы', 'ь', 'э'],
       ['я', 'а', 'б', ..., 'ь', 'э', 'ю']], dtype='<U1')

In [5]:
def vigenere_cipher(message, key, vigenere_table):
    """
    Шифрует сообщение message шифром Виженера с ключом key
    """
    # приводим сообщение к нижнему регистру, избавляемся от пробелов и знаков препинания
    mes = message.lower().replace(" ", "")
    mes = mes.translate(str.maketrans('', '', string.punctuation))

    # теперь, если нужно, удлинним ключ так, чтобы он покрывал всё сообщение
    long_key = key
    n = len(key) # длина изначального ключа

    # пока длина ключа меньше длины сообщения..
    while len(long_key) < len(mes):
        m = len(long_key)
        # добавляем к ключу символ, стоящий на n позиций раньше
        long_key = long_key + long_key[m - n] 

    # криптограмма
    message_encrypted = ""

    for i in range(len(mes)):
        # порядковый номер очередного символа открытого текста
        column = letter2number[mes[i]]
        # порядковый номер соответствующего символа ключа
        row = letter2number[long_key[i]]

        # получаем зашифрованный символ из таблицы Виженера
        # и добавляем его к криптограмме
        message_encrypted += vigenere_table[row][column]

    return message_encrypted

In [6]:
print(vigenere_cipher("криптография серьезная наука", "математика", vigenere_table))
print(vigenere_cipher("Мир - сцена, где всякий свою роль играть обязан", "венецианскийкупец", vigenere_table))

цръфюохшкффягкььчпчалнтшца
онэцмнннфонлытщняузыгжцлйщншйьпэжхйеъ


# Шифрование с помощью решеток

In [7]:
def rotare_cell(cell, k):
    """
    Поворачивает квадратную решетку cell со стороной k по часовой стрелке на 90 градусов
    """
    cell_r = cell.T # транспонируем исходную матрицу
    
    result = np.full((k, k), 'а') # результирующая решетка

    # теперь нужно инвертировать порядок столбцов;
    # перебираем все элементы в решетке..
    for i in range(k):
        for j in range(k):
            # и отражаем их относительно центрального столбца (или условной вертикальной оси)
            result[i][j] = cell_r[i][k - j - 1]

    return result

def get_holes(cell, k):
    """
    Генерирует случайное расположение трафаретных отверстий в квадрате cell размерности k,
    подходящее для дальнейшего использования шифрования с помощью решеток
    """
    # номера четырех под-решеток (далее "клеток"), из которых будут удаляться номера с 1 до k ** 2
    cell_nums = np.random.randint(0, 4, k ** 2) 

    # границы клеток
    intervals = {
        0 : [[0, k], [0, k]], 
        1 : [[0, k], [k, 2 * k]], 
        2 : [[k, 2 * k], [k, 2 * k]],
        3 : [[k, 2 * k], [0, k]]
    }

    # индексы ("координаты") трафаретных отверстий
    hole_indexes = []

    for i in range(k ** 2):
        cell_num = cell_nums[i] # номер клетки, из которой нужно удалить очередной номер 
        interval = intervals[cell_num] # границы этой клетки
        number = str(i + 1) # номер, который нужно удалить

        for j in range(interval[0][0], interval[0][1]): ######
            for l in range(interval[1][0], interval[1][1]): ## проходим по клетке
                if cell[j][l] == number: # если номер - тот, который ищем..
                    hole_indexes.append((j, l)) # добавляем его индексы в массив
                    break # и прерываем цикл

    return hole_indexes

def grille_cipher(message, key, example = False):
    """
    Шифрует сообщение message с помощью решёток с ключом key.
    Если example == True, используется трафарет из задания к лабораторной
    """
    # приводим сообщение к нижнему регистру, избавляемся от пробелов и знаков препинания
    mes = message.lower().replace(" ", "")
    mes = mes.translate(str.maketrans('', '', string.punctuation))

    # длина сообщений
    n = len(mes)

    # размер клетки
    k = math.ceil(math.ceil(np.sqrt(n)) / 2)

    # если сообщение не умещается в решетку 2k x 2k
    if (2 * k) ** 2 < n:
        k += 1 # увеличиваем k на единицу

    # если сообщение не заполняет решетку (должны остаться пустые ячейки), 
    # то пока длина сообщения не равна площади решетки..
    while len(mes) < (2 * k) ** 2:
        mes += 'а' # дописываем в конец сообщения "мусор" - букву "а"

    # 1-я клетка (левая верхняя)
    cell_1 = np.full((k, k), 0)

    # заполянем её натуральными числами сверху-вниз слева-направо
    for i in range(k):
        for j in range(k):
            cell_1[i][j] = str(i * k + j + 1)

    # следующие клетки получаем поворотом предыдущих
    cell_2 = rotare_cell(cell_1, k) # 2-я клетка (правая верхняя)
    cell_3 = rotare_cell(cell_2, k) # 3-я клетка (правая нижняя)
    cell_4 = rotare_cell(cell_3, k) # 4-я клетка (левая нижняя)

    # теперь составим из клеток одну решетку размера 2k x 2k
    cell = np.full((2 * k, 2 * k), '0')
    cell[:k, :k] = cell_1
    cell[:k, k:] = cell_2
    cell[k:, k:] = cell_3
    cell[k:, :k] = cell_4

    if example: # если параметр example - True..
        # используем трафарет из примера
        holes = [(0, 3), (2, 1), (2, 3), (3, 2)] 
    else: # иначе..
        # генерируем случайный трафарет и сортируем координаты отверстий
        # так, чтобы они шли сверху-вниз и слева-направо, т.е.
        # во-первых, по возрастанию первого индекса и во-вторых, - второго 
        holes = sorted(get_holes(cell, k), key = lambda x : (x[0], x[1]))
    
    table = np.full((2 * k, 2 * k), ' ') # таблица
    template = np.full((2 * k, 2 * k), '0') # трафарет

    # отметим положение отверстий в трафарете
    # и заодно выведем его на экран
    print("Использованный шаблон: ")
    for i in range(2 * k):
        for j in range(2 * k):
            if (i, j) in holes: # если на месте ячейки должно быть отверстие
                template[i][j] = '1' # ставим в трафарете единицу
                print('\u25A0', end = ' ') # и выводим черный квадрат
            else: # в противном случае..
                # выводим число из ранее заготовленной решетки
                print(cell[i][j], end = ' ') 
        print()

    for i in range(4): # четыре раза:
        for j in range(k ** 2): # проходим по всем отверстиям в трафарете
            # и записываем туда очередную букву
            table[holes[j][0]][holes[j][1]] = mes[i * (k ** 2) + j] 

        # поворачиваем трафарет
        template = rotare_cell(template, 2 * k)
        # и обновляем расположение отверстий
        holes = [(hole[0], hole[1]) for hole in np.array(np.where(template == '1')).T]

        # выводим результат после текущего шага
        print("Шаг №{}".format(i + 1))
        print(table)
    
    # порядковые номера букв в ключевом слове, отстортированные по возрастанию
    nums = sorted([letter2number[letter] for letter in key])

    # меняем номера обратно на буквы..
    route_order = [abc[number] for number in nums]
    # и получаем порядок, в котором нужно брать столбцы
    route_order = [key.index(letter) for letter in route_order]

    # криптограмма
    message_encrypted = ""


    for j in route_order: # проходим по столбцам в заданном порядке
        for i in range(2 * k): # проходим по всем строкам
            # добавляем очередную букву в криптограмму
            message_encrypted += table[i][j]
    
    return message_encrypted

In [8]:
print(grille_cipher("договор подписали", "шифр", example = True))

Использованный шаблон: 
1 2 3 ■ 
3 4 4 2 
2 ■ 4 ■ 
1 3 ■ 1 
Шаг №1
[[' ' ' ' ' ' 'д']
 [' ' ' ' ' ' ' ']
 [' ' 'о' ' ' 'г']
 [' ' ' ' 'о' ' ']]
Шаг №2
[[' ' ' ' ' ' 'д']
 [' ' 'в' ' ' ' ']
 ['о' 'о' ' ' 'г']
 [' ' 'р' 'о' 'п']]
Шаг №3
[[' ' 'о' ' ' 'д']
 ['д' 'в' 'п' ' ']
 ['о' 'о' ' ' 'г']
 ['и' 'р' 'о' 'п']]
Шаг №4
[['с' 'о' 'а' 'д']
 ['д' 'в' 'п' 'л']
 ['о' 'о' 'и' 'г']
 ['и' 'р' 'о' 'п']]
овордлгпапиосдои


In [9]:
print(grille_cipher("Ад пуст. Все дьяволы сюда слетелись", "буряад"))

Использованный шаблон: 
1 ■ 3 7 4 1 
4 5 ■ 8 5 2 
7 ■ ■ 9 6 3 
3 6 9 9 8 ■ 
2 5 8 6 ■ ■ 
1 4 7 ■ 2 ■ 
Шаг №1
[[' ' 'а' ' ' ' ' ' ' ' ']
 [' ' ' ' 'д' ' ' ' ' ' ']
 [' ' 'п' 'у' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'с']
 [' ' ' ' ' ' ' ' 'т' 'в']
 [' ' ' ' ' ' 'с' ' ' 'е']]
Шаг №2
[[' ' 'а' ' ' ' ' ' ' ' ']
 [' ' ' ' 'д' 'д' ' ' 'ь']
 [' ' 'п' 'у' 'я' 'в' ' ']
 ['о' ' ' ' ' ' ' ' ' 'с']
 [' ' 'л' ' ' ' ' 'т' 'в']
 ['ы' 'с' 'ю' 'с' ' ' 'е']]
Шаг №3
[['д' 'а' 'а' ' ' ' ' ' ']
 ['с' 'л' 'д' 'д' ' ' 'ь']
 ['е' 'п' 'у' 'я' 'в' ' ']
 ['о' ' ' ' ' 'т' 'е' 'с']
 [' ' 'л' ' ' 'л' 'т' 'в']
 ['ы' 'с' 'ю' 'с' 'и' 'е']]
Шаг №4
[['д' 'а' 'а' 'с' 'ь' 'а']
 ['с' 'л' 'д' 'д' 'а' 'ь']
 ['е' 'п' 'у' 'я' 'в' 'а']
 ['о' 'а' 'а' 'т' 'е' 'с']
 ['а' 'л' 'а' 'л' 'т' 'в']
 ['ы' 'с' 'ю' 'с' 'и' 'е']]
ьаветидсеоаыаьасвеадуааюалпалссдятлс
